# Загрузка Order book из CSV

In [12]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from alex.database import Database
from pprint import pprint

### Скоростная загрузка с диска, из заархивированного CSV-файла

In [2]:
# результат выполнения запроса:
# select * from tvf_get_order_book_snapshots(2880) where exchange='binance' and pair='ETH/USDT' # запрос на 20 минут
order = pd.read_csv('tvf_get_order_book_snapshots 15.09.2018-01.10.2018.zip', index_col='dt')

In [36]:
# загрузка истории
# 15 секунд
db = Database()
sql_history = f"select * from v_history with (snapshot) where exchange='binance' and pair='ETH/USDT' and dt>'2018-09-23'"
df_history = db.query(sql_history)
df_history.set_index('dt', inplace=True)

### Немного статистики по загруженным датасетам

In [6]:
order.describe()

,exchange,pair,bid_ask,price,amount,volume
count,153400,153400,153400,153400,153400,153400
unique,1,1,2,1509,22817,29503
top,binance,ETH/USDT,ask,232,1,"10,06027"
freq,153400,153400,76700,682,3122,500


In [18]:
len(order.index.sort_values().unique())

767

In [37]:
len(df_history)

1221495

### Получение Order Book в другом виде, напрямую из исторической таблицы dbo.order_book

Это не снимки, а только новые записи, добавляемые в order book в момент отрабатывания воркера. Поля ValidFrom и ValidTill показывают время жизни каждого ордера

In [59]:
# должно быть около 30 секунд
sql_orderbook = "SELECT *, ValidFrom, ValidTill FROM mem.order_book for system_time from '2018-09-23' to '2018-10-01' where id_ex_pair=19"
# id_ex_pair=19 - это Binance + ETH/USDT
df_orderbook = db.query(sql_orderbook)

In [64]:
df_orderbook['dt'] = pd.to_datetime(df_orderbook.dt)
df_orderbook['id'] = df_orderbook['id'].astype(int)
df_orderbook['ValidFrom'] = pd.to_datetime(df_orderbook.ValidFrom)
df_orderbook['ValidTill'] = pd.to_datetime(df_orderbook.ValidTill)
del df_orderbook['id_ex_pair']

In [66]:
df_orderbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317073 entries, 0 to 2317072
Data columns (total 8 columns):
id            int32
id_ex_pair    int64
dt            datetime64[ns]
is_bid        bool
price         float64
amount        float64
ValidFrom     datetime64[ns]
ValidTill     datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(2), int32(1), int64(1)
memory usage: 117.1 MB


In [67]:
# dt - время загрузки, id - порядковый номер записи, обычный инкремент identity(1,1)
df_orderbook.sort_values(['dt','id'], inplace=True)
df_orderbook.set_index(['dt','id'], inplace=True)

##### Скрипт загружает Order Book 1 раз в 3 секунды. Интервалы более 3 секунд между соседними записями могут означать, что Order Book не менялся, либо по каким-то причинам увеличилось время срабатывания скрипта

In [68]:
df_orderbook.tail(10)

id_ex_pair  is_bid   price    amount  \
dt                  id                                               
2018-09-30 23:59:55 35197647          19    True  232.90   0.22000   
                    35197648          19   False  233.61   0.04306   
                    35197649          19    True  230.50   4.07605   
                    35197650          19   False  233.51   3.77501   
                    35197651          19   False  233.25  41.12412   
2018-09-30 23:59:58 35197780          19    True  230.96   0.51331   
                    35197781          19    True  231.60   0.81217   
                    35197782          19    True  232.91   0.17800   
                    35197783          19    True  232.89  21.34600   
                    35197784          19   False  236.15   0.38710   

                                       ValidFrom           ValidTill  
dt                  id                                                
2018-09-30 23:59:55 35197647 2018-09-30 23:59:56 2018-10-01 00:00:05  
                    35197648 2018-09-30 23:59:56 2018-10-01 00:01:34  
                    35197649 2018-09-30 23:59:56 2018-10-01 00:00:18  
                    35197650 2018-09-30 23:59:56 2018-10-01 00:00:12  
                    35197651 2018-09-30 23:59:56 2018-10-01 00:00:24  
2018-09-30 23:59:58 35197780 2018-09-30 23:59:59 2018-10-01 00:00:21  
                    35197781 2018-09-30 23:59:59 2018-10-01 00:00:59  
                    35197782 2018-09-30 23:59:59 2018-10-01 00:00:05  
                    35197783 2018-09-30 23:59:59 2018-10-01 00:00:02  
                    35197784 2018-09-30 23:59:59 2018-10-01 00:00:02

In [69]:
df_orderbook.describe()

,id_ex_pair,price,amount
count,2317073.0,2.317073e+06,2.317073e+06
mean,19.0,2.254567e+02,2.116770e+01
std,0.0,1.035779e+01,8.315237e+01
min,19.0,2.016800e+02,1.000000e-05
25%,19.0,2.165800e+02,9.630000e-01
50%,19.0,2.259200e+02,4.000000e+00
75%,19.0,2.330500e+02,1.704812e+01
max,19.0,2.523100e+02,4.091701e+03


In [70]:
# кол-во записей в Order Book
len(df_orderbook)

2317073

In [76]:
# количество уникальных дат - дат загрузок Order Book
len(df_orderbook.index.unique(0))

225776